# Magenta Network

#Download dependencies
Versions compatibility taken from https://colab.research.google.com/notebooks/magenta/hello_magenta/hello_magenta.ipynb

In [ ]:
print('Installing dependencies...')
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi

!pip install -qU magenta

import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)
ctypes.util.find_library = proxy_find_library

print('Importing libraries and defining some helper functions...')
from google.colab import files

import magenta
import note_seq
import tensorflow

#Set path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('drive/My Drive/Project/')

In [ ]:
os.chdir('./magenta/scripts')

#Create Dataset

In [ ]:
!python convert_dir_to_note_sequences.py --input_dir=../models/melody_rnn/JazzDataset --output_file = ./notesequences.tfrecord --recursive

In [ ]:
!python melody_rnn_create_dataset.py --config=attention_rnn --input=notesequences.tfrecord --output_dir=./dataset_output --eval_ratio=0.10

#Training

In [ ]:
os.chdir('../models/melody_rnn')

In [ ]:
!python melody_rnn_train.py --config=attention_rnn --run_dir=./checkpoint --sequence_example_file=./dataset_output/training_melodies.tfrecord hparams="batch_size=64,rnn_layer_sizes=[128,128]" --num_training_steps=100

#Generate music after Training

In [ ]:
!python melody_rnn_generate.py --config=attention_rnn --run_dir=./logdir --output_dir=./generatedAfterTraining --num_outputs=10 --num_steps=128 --hparams="batch_size=64,rnn_layer_sizes=[128,128]"